# 💡 MosaicPi_Guide: **"Linear Algebra in MosaicPi"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**MosaicPi中的线性代数应用**

This notebook introduces the **fundamental concepts of linear algebra** that are widely used in applied mathematics, data science, physics, and engineering. The focus is on building an intuitive understanding alongside practical computations using NumPy.

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: MosaicPi, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*

---

---

# 6. 广义逆与最小二乘（Pseudoinverse & Least Squares）

## 6.1 Moore–Penrose 伪逆 $A^+$

* **SVD 定义**：$A\in\mathbb{R}^{m\times n}$, $A=U\Sigma V^\top$
  $\Sigma=\mathrm{diag}(\sigma_1\ge\cdots\ge\sigma_r>0)$

  $$
  A^+ \;=\; V\,\Sigma^+\,U^\top,\quad 
  \Sigma^+=\mathrm{diag}\Big(\tfrac{1}{\sigma_1},\dots,\tfrac{1}{\sigma_r}\Big)
  $$
* **Penrose 四条件**（表征唯一性）：

  $$
  AA^+A=A,\ \ A^+AA^+=A^+,\ \ (AA^+)^\top=AA^+,\ \ (A^+A)^\top=A^+A
  $$
* **投影性质**：
  $AA^+$ 为到 $\mathcal{R}(A)$ 的正交投影；$A^+A$ 为到 $\mathcal{R}(A^\top)$ 的正交投影。

---

## 6.2 线性最小二乘（LLS）

### 6.2.1 过约束（$m>n$）：$\min_x \|Ax-b\|_2$

* **SVD 公式**：$x^\* = A^+ b = \sum_{i=1}^r \frac{u_i^\top b}{\sigma_i} v_i$
* **满列秩时**（数值不推荐但可写）：
  正规方程 $A^\top A x=A^\top b \Rightarrow x=(A^\top A)^{-1}A^\top b$
* **数值推荐**：QR 或 SVD（稳健、可察觉秩缺陷）

### 6.2.2 欠约束（$m<n$）：$\min\|x\|_2$ s.t. $Ax=b$

* **最小范数解**：$x^\* = A^+ b$
  若满行秩：$x^\* = A^\top (AA^\top)^{-1} b$

### 6.2.3 加权/稳健/正则

* **加权最小二乘（WLS）**：$\min\|W^{1/2}(Ax-b)\|_2\Rightarrow x=(A^\top W A)^{-1}A^\top W b$
* **岭回归（Tikhonov）**：$\min\|Ax-b\|_2^2+\lambda^2\|x\|_2^2$
  $\Rightarrow x=(A^\top A+\lambda^2 I)^{-1}A^\top b$
  **SVD 视角**：系数滤波 $\frac{\sigma_i}{\sigma_i^2+\lambda^2}$
* **截断 SVD（TSVD）**：仅保留 $\sigma_i\ge\tau$ 的分量（去噪/降秩）
* **总最小二乘（TLS）**：当 $A$ 与 $b$ 都含噪时更合适（正交回归）

---

## 6.3 条件数与稳定性

* **谱条件数**：$\kappa_2(A)=\sigma_{\max}/\sigma_{\min}$（小奇异值导致解放大）
* **秩判据**：阈值 $\tau\approx \sigma_{\max}\cdot \epsilon\cdot \max(m,n)$ 决定“有效秩”
* **缩放与标准化**：变量尺度差异会劣化条件数（建议 z-score/物理量纲归一）

---

## 6.4 与隐函数/雅可比的连接（数据驱动 IFT）

### 6.4.1 局部线性化与雅可比估计

* **从数据估 $J$**：在点 $(x_0,y_0)$ 邻域采样 $(\Delta x_i,\Delta y_i)$

  $$
  \Delta Y \approx J\,\Delta X\quad \Rightarrow\quad \hat J = \Delta Y\,(\Delta X)^+
  $$

  * 过约束：最小二乘意义下的 $J$
  * 欠约束：得到最小范数 $J$
  * 加权版：$\hat J=\Delta Y\,(\Delta X)^{+}_W$

### 6.4.2 由隐函数线性化求响应

* **隐函数** $F(x,y)=0$ 的一阶线性化：
  $J_y \Delta y = -J_x \Delta x$
* **解法**（视可解性/噪声）：

  * 满列秩 $J_y$：$\Delta y = -J_y^{-1} J_x \Delta x$
  * 一般情形：$\Delta y = -J_y^+ J_x \Delta x$（**最小范数**或**最小残差**意义）
  * 不适定/噪声：$\Delta y = -J_{y,\lambda}^+ J_x \Delta x$（岭/TSVD 稳定化）
* **沿方向灵敏度**：给定方向 $d$，$\partial y/\partial s|_{d} \approx -J_y^+ J_x d$

### 6.4.3 排名与可逆域判定

* **秩与可逆性**：$\mathrm{rank}(J_y)=p$（y 维度）是局部显函数存在的必要条件
* **条件数**：$\kappa(J_y)$ 评估可逆域的**敏感性/稳定性**
* **小奇异值**：提示局部“近退化”，IFT 虽可用但误差/增益巨大（需正则）

---

## 6.5 实践流程（推荐操作）

1. **预处理**：去均值/标准化、单位一致性、异常值筛查
2. **采样几何**：在目标点周围设计小扰动（正交设计/LHS），保证 $\Delta X$ **满秩且良态**
3. **SVD 估计**：$\hat J=\Delta Y(\Delta X)^+$，并记录 $\sigma_i$、$\kappa$、有效秩
4. **稳定化**：若 $\kappa$ 大，采用岭或 TSVD；必要时用 WLS/TLS
5. **误差评估**：交叉验证残差、对比有限差分/AD 的梯度、一致性检查
6. **隐函数响应**：$\Delta y=-J_y^+J_x\Delta x$，并随 $\|\Delta x\|$ 控制线性化半径
7. **报告**：给出 $\mathrm{rank}(J_y)$、$\kappa(J_y)$、阈值、所用正则参数与重构误差

---

## 6.6 常见陷阱与对策

* **共线/近共线**：导致 $\sigma_{\min}\to 0$；→ 重新设计扰动方向、变量缩放、正则化
* **异方差/相关噪声**：→ WLS / GLS（协方差已知时）
* **双边噪声**：→ TLS / Total Least Squares
* **外点**：→ Huber/Huberized LS、RANSAC、稳健 PCA 预清洗
* **量纲不一致**：→ 标准化/物理无量纲化后再做 LLS/伪逆

---

## 6.7 公式速查

* **过约束最小二乘**：$x^\*=A^+b$
* **欠约束最小范数**：$x^\*=A^+b$
* **岭解**：$x=(A^\top A+\lambda^2 I)^{-1}A^\top b$
* **雅可比估计**：$\hat J=\Delta Y(\Delta X)^+$
* **隐函数线性响应**：$\Delta y=-J_y^+J_x\Delta x$

---
